In [1]:
import os
import orchest
import yaml

In [2]:
postgres_service = orchest.get_service("postgres"); postgres_service 

{'name': 'postgres',
 'image': 'postgres',
 'command': '',
 'env_variables': {'POSTGRES_DB': 'jaffle_shop',
  'POSTGRES_HOST_AUTH_METHOD': 'trust'},
 'env_variables_inherit': [],
 'internal_hostname': 'internal-service-postgres-5c9a40f9-7b21ba4a',
 'external_urls': {},
 'base_paths': {}}

In [3]:
with open('profiles.yml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    dbt_profile = yaml.load(file, Loader=yaml.FullLoader)

In [4]:
dbt_profile

{'jaffle_shop': {'target': 'dev',
  'outputs': {'dev': {'type': 'postgres',
    'host': '<host>',
    'user': '<user>',
    'password': '<password>',
    'port': 5432,
    'dbname': 'jaffle_shop',
    'schema': 'dbt_alice',
    'threads': 4}}}}

In [5]:
dbt_profile['jaffle_shop']['outputs']['dev']['host'] = postgres_service['internal_hostname']
dbt_profile['jaffle_shop']['outputs']['dev']['user'] = 'postgres'
dbt_profile['jaffle_shop']['outputs']['dev']['password'] = ''

In [6]:
dbt_profile

{'jaffle_shop': {'target': 'dev',
  'outputs': {'dev': {'type': 'postgres',
    'host': 'internal-service-postgres-5c9a40f9-7b21ba4a',
    'user': 'postgres',
    'password': '',
    'port': 5432,
    'dbname': 'jaffle_shop',
    'schema': 'dbt_alice',
    'threads': 4}}}}

In [7]:
dbt_profile_path = os.path.join(os.environ["HOME"], ".dbt", "profiles.yml"); dbt_profile_path

'/home/jovyan/.dbt/profiles.yml'

In [8]:
os.makedirs(os.path.split(dbt_profile_path)[0], exist_ok=True)
with open(dbt_profile_path, 'w') as fp:
    yaml.dump(dbt_profile, fp)

In [9]:
!dbt debug

13:27:17  Running with dbt=1.0.1
dbt version: 1.0.1
python version: 3.7.6
python path: /opt/conda/bin/python3.7
os info: Linux-5.4.0-92-generic-x86_64-with-debian-bullseye-sid
Using profiles.yml file at /home/jovyan/.dbt/profiles.yml
Using dbt_project.yml file at /project-dir/dbt_project.yml

Configuration:
  profiles.yml file [OK found and valid]
  dbt_project.yml file [OK found and valid]

Required dependencies:
 - git [OK found]

Connection:
  host: internal-service-postgres-5c9a40f9-7b21ba4a
  port: 5432
  user: postgres
  database: jaffle_shop
  schema: dbt_alice
  search_path: None
  keepalives_idle: 0
  sslmode: None
  Connection test: [OK connection ok]

All checks passed!


In [10]:
!dbt seed

13:27:31  Running with dbt=1.0.1
13:27:31  Partial parse save file not found. Starting full parse.
13:27:32  Found 5 models, 20 tests, 0 snapshots, 0 analyses, 165 macros, 0 operations, 3 seed files, 0 sources, 0 exposures, 0 metrics
13:27:32  
13:27:32  Concurrency: 4 threads (target='dev')
13:27:32  
13:27:32  1 of 3 START seed file dbt_alice.raw_customers.................................. [RUN]
13:27:32  2 of 3 START seed file dbt_alice.raw_orders..................................... [RUN]
13:27:32  3 of 3 START seed file dbt_alice.raw_payments................................... [RUN]
13:27:33  1 of 3 OK loaded seed file dbt_alice.raw_customers.............................. [INSERT 100 in 0.85s]
13:27:33  2 of 3 OK loaded seed file dbt_alice.raw_orders................................. [INSERT 99 in 0.85s]
13:27:33  3 of 3 OK loaded seed file dbt_alice.raw_payments............................... [INSERT 113 in 0.85s]
13:27:33  
13:27:33  Finished running 3 seeds in 0.97s.
13:27:33  


In [11]:
!dbt run

13:27:41  Running with dbt=1.0.1
13:27:41  Found 5 models, 20 tests, 0 snapshots, 0 analyses, 165 macros, 0 operations, 3 seed files, 0 sources, 0 exposures, 0 metrics
13:27:41  
13:27:41  Concurrency: 4 threads (target='dev')
13:27:41  
13:27:41  1 of 5 START view model dbt_alice.stg_customers................................. [RUN]
13:27:41  2 of 5 START view model dbt_alice.stg_orders.................................... [RUN]
13:27:41  3 of 5 START view model dbt_alice.stg_payments.................................. [RUN]
13:27:41  2 of 5 OK created view model dbt_alice.stg_orders............................... [CREATE VIEW in 0.16s]
13:27:41  3 of 5 OK created view model dbt_alice.stg_payments............................. [CREATE VIEW in 0.16s]
13:27:41  1 of 5 OK created view model dbt_alice.stg_customers............................ [CREATE VIEW in 0.16s]
13:27:41  4 of 5 START table model dbt_alice.orders....................................... [RUN]
13:27:41  5 of 5 START table mod